In [ ]:
%%writefile sender.py
import sys
import os
import math

# store arguments
input_file = sys.argv[1];
Source_IP = sys.argv[2];
Destination_IP = sys.argv[3];
Source_port = sys.argv[4];
Destination_port = sys.argv[5];
output_file = sys.argv[6]

# print 
print("Source Port:",Source_IP)
print("Desternation Port:", Destination_IP)

# print source ip (ex.192.168.1.100)
print("Source IP:")
s_ip = Source_IP.split('.')
for i in range(4):
    print("source ip", i+1,end =":")
    print(s_ip[i])
    print("source IP to hex:",hex(int(s_ip[i])))
print()

# print destination ip (192.168.35.10)
print("Destination IP:")
d_ip = Destination_IP.split('.')
for i in range(4):
    print("Destination", i+1,end =":")
    print(d_ip[i])
    print("Destination IP to hex:",hex(int(d_ip[i])))

print()

#print Source port

print("Source_port",Source_port)
print("Source_port hex formate:",hex(int(Source_port)))
print()

#print destination port

print("Destination_port",Destination_port)
print("Destination_port hex formate:",hex(int(Destination_port)))
print()


#get UDP length(the length of data from input file + 8 bytes)

with open(input_file,'rb') as data_file:
  contents_data = data_file.read()
  print("input file:", contents_data)
  print("without zero padding:", len(contents_data))
  UDP_length = len(contents_data) + 8
  print("UDP total length:",  UDP_length)
print("Constents data: ", list(contents_data))

# get key
with open('Keys0','rb') as key_file:
  contents_key = list(key_file.read())
  print("key:", contents_key)
print()

#psedudo header list 
data = []
###source ip
data.extend([int(x) for x in s_ip])
#print("data1: ", data)
###destination ip
data.extend([int(x) for x in d_ip])
###zeros and protocal
data.extend([0,17])
###total length
data.extend([UDP_length//256, UDP_length%256])
#data.append(hex(int(UDP_length)))


# UDP head
###source port
data.extend([int(Source_port)//256, int(Source_port)%256])
###destination port
data.extend([int(Destination_port)//256, int(Destination_port)%256])
###total length
data.extend([UDP_length//256, UDP_length%256])

print("Input data list:", data)
#------------------- encrption part----------------------
def str_to_hex(str):
  s = str.encode('latin-1')
  return s.hex()

# swap L & R and then do xor between L and key
def res_Lxor(L,R,KeyC):
  L_prime = R
  R_prime = L^KeyC
  return L_prime,R_prime
  

# key_str: list of ascii number of keys
def en(data,key_list):
  print("Key List:", key_list)
  if len(data) %2 == 1:
    data.append(0)
  ans = []
  for i in range(0,len(data),2):
    tmpL, tmpR = data[i], data[i+1]
    for key in key_list:
      tmpL,tmpR = res_Lxor(tmpL,tmpR,key)
    ans.extend([tmpL,tmpR])
  return ans

#----------------------done --------------------------------
en_input = en(list(contents_data), contents_key)

data.extend(en_input)
# print("eninputtttttttttttttttttttt",en_input)
# print("fuck youuuuuuuuuuuuuuuuuuuu",''.join(map(str,en_input)))
# msg_string = ''.join(map(str,en_input))
# fuck = [ord(i) for i in msg_string]
# print("character list of input",fuck)
sum_var = 0 

for i in range(len(data)//2):
  sum_var += data[2*i]*256 + data[2*i+1]
print("sum in:", sum_var)

while sum_var >= 65536:
  last_4bits = sum_var & 0xffff
  left_bits = sum_var >> 16
  sum_var = last_4bits + left_bits
  
checksum = ~sum_var & 0xffff
data = data[12:18] + [checksum//256, checksum%256] + data[18:]
print("Data:",[hex(i) for i in data])
#checksum &= 0xffff

print("checksum:", hex(checksum))

with open(output_file,"wb") as f:
  f.write(bytes(data))

print("File is successfully written to datagram")



# get new file and save data in new file for checking size of file 
fd = open("sender_datagram", "wb")
fd.write(bytes(data))
fd.close()

file_size = os.path.getsize('sender_datagram')
print("File Size is :", file_size, "bytes")



Overwriting sender.py


In [ ]:
#!python3 sender.py oddchars 192.168.52.4 192.168.35.10 2300 23540 datagram

In [ ]:
!python3 sender.py test.txt 192.168.52.4 192.168.35.10 2300 23540 datagram

Source Port: 192.168.52.4
Desternation Port: 192.168.35.10
Source IP:
source ip 1:192
source IP to hex: 0xc0
source ip 2:168
source IP to hex: 0xa8
source ip 3:52
source IP to hex: 0x34
source ip 4:4
source IP to hex: 0x4

Destination IP:
Destination 1:192
Destination IP to hex: 0xc0
Destination 2:168
Destination IP to hex: 0xa8
Destination 3:35
Destination IP to hex: 0x23
Destination 4:10
Destination IP to hex: 0xa

Source_port 2300
Source_port hex formate: 0x8fc

Destination_port 23540
Destination_port hex formate: 0x5bf4

input file: b'---------------------------------------------------\r\nMessage from Client:\r\n00/00/0000 00:00\\:00 IP:000.00.0.00\r\n---------------------------------------------------\r\nMessage from Client:\r\n00/00/0000 00:00\\:00 IP:000.00.0.00\r\n---------------------------------------------------\r\nMessage from Client:\r\n00/00/0000 00:00\\:00 IP:000.00.0.00\r\n---------------------------------------------------\r\nMessage from Client:\r\n00/00/0000 00:00\\:

In [ ]:
 #!python3 sender.py image.jpg 192.168.1.100 192.168.1.101 12345 23456 datagram

In [ ]:
%%writefile Receiver.py
import sys
import math
import os
import binascii

# get key
with open('Keys0','rb') as key_file:
  contents_key = key_file.read()
  print("key:", contents_key)
print()

#open the datagram from sender
datagram = sys.argv[3]
data_open = open(datagram,'rb')
data = list(data_open.read())
print("data",data)
print("length of data:", len(data))


#Source & Desternation IP
Source_IP = sys.argv[1]
print("Source Port:",Source_IP)
Destination_IP = sys.argv[2]
print("Desternation Port:", Destination_IP)
print()

print("Source IP:")
s_ip = Source_IP.split('.')
for i in range(4):
    print("source ip", i+1,end =":")
    print(s_ip[i])
    print("source IP to hex:",hex(int(s_ip[i])))
print()

# print destination ip (ex. 192.168.35.10)
print("Destination IP:")
d_ip = Destination_IP.split('.')
for i in range(4):
    print("Destination", i+1,end =":")
    print(d_ip[i])
    print("Destination IP to hex:",hex(int(d_ip[i])))

print()


re_data = []


###source ip
re_data.extend([int(x) for x in s_ip])
###destination ip
re_data.extend([int(x) for x in d_ip])
###zeros and protocal
re_data.extend([0,17])

###total length
UDP_length = data[4:6]
print("UDP_length,",UDP_length)
re_data.extend(UDP_length)

# UDP head
# ###source port
Source_port = data[:2]
# print('Source port:', type(Source_port))
print('Source port:', Source_port)


# ###destination port
Destination_port = data[2:4]
print('Source port:', Destination_port)


###total length
re_data.extend(data)
print("Re-data:", [hex(i) for i in re_data])


#-----------------------------------------------------------------
##checksum
checksum = data[12:16]
print("input data list with checksum:", re_data)
print("input data list length", len(re_data))

sum_var = 0 
data = re_data

for i in range(len(data)//2):
  sum_var += data[2*i]*256 + data[2*i+1]
print("sum in:", sum_var)

sum_var = 0 
for i in range(len(data)//2):
  sum_var += data[2*i]*256 + data[2*i+1]
print("sum in:", sum_var)

while sum_var >= 65536:
  last_4bits = sum_var & 0xffff
  left_bits = sum_var >> 16
  sum_var = last_4bits + left_bits
  
checksum = sum_var

print('checksum', hex(checksum))
if checksum != 65535:
  print("Checksum error!!!!!!")
  
print("Checksum in decimal:", checksum)
print("Checksum in hex:",hex(checksum))
print()

#----------------Decryption----------------------------
#decrption
# swap L & R and then do xor between L and key
def res_Lxor(L,R,KeyC):
  L_prime = R
  R_prime = L^KeyC
  return L_prime,R_prime
  
# key_str: list of ascii number of keys
def de(data,key_list):
  print("Key List:", key_list)
  if len(data) %2 == 1:
    data.append(0)
  ans = []
  for i in range(0,len(data),2):
    tmpL, tmpR = data[i], data[i+1]
    for key in key_list:
      tmpL,tmpR = res_Lxor(tmpL,tmpR,key)
    ans.extend([tmpL,tmpR])
  return ans

#------------------done-------------------------------

m = data[20:]
de_m = de(m, contents_key)
print("sdjfkasldfjasldfjaslfjdasl;jdfadf",de_m)
de_m = de_m[:-1] if data[11]%2 == 1 else de_m
# print("de-m2,", ''.join([chr(i) for i in de_m]))
final_de  =  ''.join([chr(i) for i in de_m])

#combination hex together 
def append_port(a, b):
  return (a*256 +b)

Sour_port = append_port(int(Source_port[0]), int(Source_port[1]))
Destin_port = append_port(int( Destination_port[0]), int( Destination_port[1]))


with open(datagram ,"w",encoding = 'latin-1') as f:
  if (checksum == 65535):
    print("Datagram from source ip:", Source_IP)
    print("Datagram from source port:",Sour_port)
    print("Destination ip:", Destination_IP)
    print("Datagram from source port:", Destin_port)
    # print("Length of data:",len(data), "bytes")
    print("Decrption output in datagram:", final_de,end='')
    f.writelines(final_de)

# if(checksum != 65535):
#     print("checksum error!!!!!!")

# fd = open(datagram, "wb")
# fd.write(bytes(final_de.encode('utf-8')))
# fd.close()

# file_size = os.path.getsize(datagram)
# print("File Size is :", file_size, "bytes")

    

Overwriting Receiver.py


In [ ]:
!python3 Receiver.py 192.168.52.4 192.168.35.10  datagram

key: b'asdfghjk'

data [8, 252, 91, 244, 2, 86, 209, 151, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 27, 2, 91, 109, 101, 123, 119, 111, 115, 40, 112, 122, 121, 101, 54, 75, 122, 97, 115, 102, 98, 50, 27, 2, 38, 56, 57, 56, 38, 39, 38, 56, 38, 56, 54, 56, 38, 50, 38, 56, 74, 50, 38, 56, 54, 65, 70, 50, 38, 56, 38, 38, 38, 56, 56, 56, 56, 56, 38, 5, 28, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 59, 37, 27, 2, 91, 109, 101, 123, 119, 111, 115, 40, 112, 122, 121, 101, 54, 75, 122, 97, 115, 102, 98, 50, 27, 2, 38, 56, 57, 56, 38, 39, 38, 56, 38, 56, 54, 56, 38, 50, 38, 56, 74, 50, 38, 56, 54, 65, 70, 50, 38, 56, 38, 38, 38, 56, 56, 56, 56, 56, 38, 5, 28, 37, 59, 37, 59, 37, 59, 37